In [19]:
from carbon.sdk import CarbonSDK, Tokens as T
#from carbon import CarbonOrderUI, CarbonSimulatorUI
from carbon.helpers.widgets import CheckboxManager, DropdownManager, PcSliderManager
from matplotlib import pyplot as plt
import numpy as np
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSDK))
!node --version

CarbonSDK v0.9 (11/Mar/2023)
v18.14.2


# Single Trades [NB31]

_allows to choose a market, retrieves market information, and then submits a trade_

In order to start the server, please run

    source /Volumes/Private/bin/sdkserver
    node server/sdkserver.mjs

In [20]:
SDK = CarbonSDK(disclaimer=False, verbose=False, Tokens=T)
pairs, pairs_s = SDK.pairs(inclstr=True)
SDK.version().get("msg")

'Carbon SDK v0.9 (7/Mar/2022)'

## Pair selection

In [21]:
try:
    pairs_sel = [s.strip() for s in pairs_s.split(",")]
    pairs_w()
except:
    pairs_w = DropdownManager(pairs_sel, defaultval="USDC/ETH")
    pairs_w()

Dropdown(options=(('USDC/ETH', 'USDC/ETH'), ('DAI/USDC', 'DAI/USDC')), value='USDC/ETH')

In [22]:
try:
    inverse_w()
except:
    inverse_w = CheckboxManager(["inverse"], values=[1])
    inverse_w()

In [23]:
tknb,tknq = pairs_w.value.split("/")
if inverse_w.values[0]:
    tknb,tknq = tknq, tknb
pair = f"{tknb}/{tknq}"
pair

'ETH/USDC'

## Market information

In [24]:
m = SDK.mGetMarketByPair(pair=pair)
m

MarketByPair(pair='ETH/USDC', bestBid=1989.9999997673926, bestAsk=2005.0000000000175, liqBid=4264.65575, liqAsk=10.74014013035547, lastBid=899.9999997186933, lastAsk=3000.000000000071)

In [25]:
print(f"bid/mid/ask = {SDK.roundsd(m.bestBid,6)}/{SDK.roundsd(m.mid,6)}/{SDK.roundsd(m.bestAsk,6)}")
print(f"liquidity = {SDK.roundsd(m.liqBid,4)} {m.liqBidUnit} bid, {SDK.roundsd(m.liqAsk,4)} {m.liqAskUnit} ask")

bid/mid/ask = 1990.0/1997.5/2005.0
liquidity = 4265.0 USDC bid, 10.74 ETH ask


## Trading settings

In [26]:
m.cpair

P('ETH/USDC')

In [27]:
try:
    bq_w()
except:
    bq_w = DropdownManager(["Base Token", "Quote Token"], defaultval="Base Token")
    bq_w()
tkn = m.cpair.tknb if bq_w.value == "Base Token" else m.cpair.tknq

Dropdown(options=(('Base Token', 'Base Token'), ('Quote Token', 'Quote Token')), value='Base Token')

In [28]:
try:
    buysell_w()
except:
    buysell_w = DropdownManager(["Buy", "Sell"], defaultval="Sell")
    buysell_w()
buysell = SDK.SELL if buysell_w.value == "Sell" else SDK.BUY

Dropdown(index=1, options=(('Buy', 'Buy'), ('Sell', 'Sell')), value='Sell')

In [29]:
print(f"Your choice: {buysell_w.value} {tkn} against {m.cpair.other(tkn)} [market = {m.pair}]")

Your choice: Sell ETH against USDC [market = ETH/USDC]


In [30]:
amount = 0.01

#### `mPrepareTrade`

In [31]:
trade = SDK.mPrepareTrade(pair, buysell, tkn, amount)
trade

TradeData(pair='ETH/USDC', tkn='ETH', traderBuySell='SELL', amount=0.01, sourceToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), targetToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), tradeByTargetAmount=False, tradeActions=[{'strategyId': 10, 'amount': 9999999497389291}], actionsWei=[{'id': 10, 'input': 9999999497389291, 'output': 19836741}], actionsTokenRes=[{'id': 10, 'sourceAmount': 0.00999999949738929, 'targetAmount': 19.836741}], tradeData={'totalSourceAmount': 0.00999999949738929, 'totalTargetAmount': 19.806985, 'effectiveRate': 1980.6985995520326})

In [32]:
trade.actionsTokenRes

[{'id': 10, 'sourceAmount': 0.00999999949738929, 'targetAmount': 19.836741}]

#### `mComposeTradeTransaction`

In [33]:
tx = SDK.mComposeTradeTransaction(**trade.cttkwargs)
tx

{'data': '0xf1c5e014000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000000000000011001ee0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000002386f251cbc4eb',
 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'},
 'value': {'type': 'BigNumber', 'hex': '0x2386f251cbc4eb'}}

#### `signsubmittx`

In [34]:
try:
    submit_w()
except:
    submit_w = CheckboxManager(["submit"], values=[0])
    submit_w()

In [35]:
if submit_w.values[0]: 
    r = SDK.signsubmittx(tx, sign=True)
else:
    r = "check box above to submit"
r

{'txReceipt': {'hash': '0x3cab6d2a63a4d71f07a247756e2b6bf59cb9a10026789208d034d39a3cc7f5c9',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}